# **RNN Serving**

The following section we create a new model serving function which wraps our class , and specify model and other resources. <br>
Deploying the serving function will provide us an http endpoint that can handle requests in real time. <br>
This function is part of the [stock-analysis demo](https://github.com/mlrun/demos/tree/master/stock-analysis).<br>
To see how the model is trained or how the data-set is generated, check out code folder in the demo repository.

### **Steps**

1. [Setup function parameters](#Setup-function-parameters)
2. [Importing the function](#Importing-the-function)
3. [Testing the function locally](#Testing-the-function-locally)
4. [Testing the function remotely](#Testing-the-function-remotely)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Following packages are required, make sure to install
# !pip install pip install torch==1.6.0
# !pip install tensorflow
# !pip install keras

### **Setup function parameters**

In [3]:
# Setting up models path
rnn_model_path = 'https://s3.wasabisys.com/iguazio/models/function-marketplace-models/rnn_serving/rnn_model.h5'
data_path = 'https://s3.wasabisys.com/iguazio/data/function-marketplace-data/rnn_serving/stocks_data.pkl'

### **Importing the function**

In [4]:
import mlrun
mlrun.set_environment(project='function-marketplace')

# Importing the function from the hub
fn = mlrun.import_function("hub://rnn_serving")
fn.apply(mlrun.auto_mount())

# Manually specifying needed packages 
fn.spec.build.commands = ['pip install torch==1.6.0', 'pip install tensorflow', 'pip install keras']

# Adding the model 
fn.add_model(key='rnn_model', model_path=rnn_model_path ,class_name='RNN_Model_Serving')

> 2021-10-17 10:43:46,363 [info] loaded project function-marketplace from MLRun DB


### **Testing the function locally**

In [5]:
# When mocking, class has to be present
from rnn_serving import *

# Mocking function
server = fn.to_mock_server()

> 2021-10-17 10:43:54,256 [info] model rnn_model was loaded
> 2021-10-17 10:43:54,257 [info] Initializing endpoint records
> 2021-10-17 10:43:54,276 [info] Loaded ['rnn_model']


In [6]:
# Getting the data
import cloudpickle as cp
from urllib.request import urlopen

rnn_data = cp.load(urlopen(data_path))

model used in this example take inputs with the shape `(None, None, 11)`. <br>
whereas the first dimenstion is the number of instances, the second dimenstion is the number of timestamps<br>
and the last dimenstion is the number of features the dataset have.<br>
our testing dataset has `(1,10,11)` means one instance to predict, with sequence length of 10, each step has 11 features.

In [8]:
import requests

# KFServing protocol event
event_data = {"inputs": rnn_data}

response = server.test(path='/v2/models/rnn_model/predict',body=event_data)
response

{'id': '1bf6a3dc4d204e6e8bfd5834f5d691f1',
 'model_name': 'rnn_model',
 'outputs': '[[0.43563252687454224]]'}

### **Testing the function remotely**

In [9]:
address = fn.deploy()

> 2021-10-17 10:43:57,192 [info] Starting remote function deploy
2021-10-17 10:43:57  (info) Deploying function
2021-10-17 10:43:57  (info) Building
2021-10-17 10:43:57  (info) Staging files and preparing base images
2021-10-17 10:43:57  (info) Building processor image
2021-10-17 10:43:58  (info) Build complete
2021-10-17 10:44:10  (info) Function deploy complete
> 2021-10-17 10:44:11,677 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-function-marketplace-rnn-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev39.lab.iguazeng.com:30255']}


In [10]:
import json
import requests

# using requests to predict
response = requests.put(address+"/v2/models/rnn_model/predict", json = json.dumps(event_data))
json.loads(response.text)

{'id': '1bf6a3dc4d204e6e8bfd5834f5d691f1',
 'model_name': 'rnn_model',
 'outputs': '[[0.43563252687454224]]'}

[Back to the top](#RNN-Serving)